In [1]:
# import xgboost as xgb
# import pandas as pd
# import numpy as np
# import pickle
# import sys
# import matplotlib.pyplot as plt
# from sklearn.metrics import mean_absolute_error, make_scorer
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import GridSearchCV
# from scipy.sparse import csr_matrix, hstack
# from sklearn.model_selection import train_test_split, KFold
# from xgboost import XGBRegressor

# import warnings
# warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
    
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import cross_val_score

from copy import deepcopy

In [2]:
train = pd.read_csv('examples/Allstate-Claims-Severity/train.csv')
test = pd.read_csv('examples/Allstate-Claims-Severity/test.csv')

In [3]:
train.shape

(188318, 132)

In [4]:
print('first 20 cols:', list(train.columns[:20]))
print('last 20 cols:', list(train.columns[-20:]))

first 20 cols: ['id', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat19']
last 20 cols: ['cat112', 'cat113', 'cat114', 'cat115', 'cat116', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14', 'loss']


In [5]:
train.describe()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
count,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000,188318.000000
mean,294135.982561,0.493861,0.507188,0.498918,0.491812,0.487428,0.490945,0.484970,0.486437,0.485506,0.498066,0.493511,0.493150,0.493138,0.495717,3037.337686
std,169336.084867,0.187640,0.207202,0.202105,0.211292,0.209027,0.205273,0.178450,0.199370,0.181660,0.185877,0.209737,0.209427,0.212777,0.222488,2904.086186
min,1.000000,0.000016,0.001149,0.002634,0.176921,0.281143,0.012683,0.069503,0.236880,0.000080,0.000000,0.035321,0.036232,0.000228,0.179722,0.670000
25%,147748.250000,0.346090,0.358319,0.336963,0.327354,0.281143,0.336105,0.350175,0.312800,0.358970,0.364580,0.310961,0.311661,0.315758,0.294610,1204.460000
50%,294539.500000,0.475784,0.555782,0.527991,0.452887,0.422268,0.440945,0.438285,0.441060,0.441450,0.461190,0.457203,0.462286,0.363547,0.407403,2115.570000
75%,440680.500000,0.623912,0.681761,0.634224,0.652072,0.643315,0.655021,0.591045,0.623580,0.566820,0.614590,0.678924,0.675759,0.689974,0.724623,3864.045000
max,587633.000000,0.984975,0.862654,0.944251,0.954297,0.983674,0.997162,1.000000,0.980200,0.995400,0.994980,0.998742,0.998484,0.988494,0.844848,121012.250000


In [6]:
pd.isnull(train).values.any()

False

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 132 entries, id to loss
dtypes: float64(15), int64(1), object(116)
memory usage: 189.7+ MB


In [12]:
# float64(15)是连续特征,int64(1),object(116)为离散特征

cat_features = list(train.select_dtypes(include=['object']).columns)
len(cat_features)

116

In [14]:
cont_features = [
    cont for cont in list(
        train.select_dtypes(include=['float64', 'int64']).columns)
    if cont not in ['id', 'loss']
]
len(cont_features)

14

In [15]:
id_col = list(train.select_dtypes(include=['int64']).columns)
id_col

['id']

In [8]:
cat_uniques=[]
for cat in cat_